# Model Output Evaluation Notebook

This notebook runs LLM inference to predict HPO terms, compares them to ground truth phenopackets, and produces a summary report.


## Step 0. Imports, Path Discovery & Sanity Checks

Load all dependencies, discover the dataset CSV automatically, and validate critical directories.


In [101]:
# Basic Setup
import sys, os, glob, json, subprocess, pickle, datetime, hashlib, warnings, random
from pathlib import Path
import pandas as pd
from typing import List, Dict, Any
from ollama import chat
from docling.document_converter import DocumentConverter, ConversionError
from pypdfium2._helpers.misc import PdfiumError
from google.protobuf.json_format import ParseDict, ParseError
from phenopackets import Phenopacket as ProtoPhenopacket
from json.decoder import JSONDecodeError

# Need this at least once for some reason:
# import .autonotebook
# from .autonotebook import tqdm as notebook_tqdm

# Make sure our utils folder is on PYTHONPATH
project_root        = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
src_folder          = os.path.join(project_root, "src")
utils_folder        = os.path.join(project_root, "notebooks", "utils")

print("Project Start:       %s" % project_root)
print("Source Folder:       %s" % src_folder)
print("Utilities Folder:    %s" % utils_folder)

for path in (src_folder, utils_folder):
    if not os.path.isdir(path):
        raise FileNotFoundError(f"Expected folder on PYTHOPATH : {path}")
    if path not in sys.path:
        sys.path.insert(0, path)

print("PYTHONPATH patched with:", src_folder, utils_folder)

try:
    from phenopacket import Phenopacket
    from report import Report
    from evaluation import PhenotypeEvaluator
except ImportError as e:
    raise ImportError(f"Could not import project utils: {e}")

# define all key paths
pdf_input_directory                 = os.path.join(src_folder, "P5", "scripts", "data", "tmp", "phenopacket_store", "pmid_pdfs")            # scripts/data/tmp/phenopacket_store/pmid_pdfs/
ground_truth_notebooks_directory    = os.path.join(src_folder, "P5", "scripts", "data","tmp", "phenopacket_store","notebooks")              # scripts/data/tmp/phenopacket_store/notebooks/
dataset_csv_path                    = os.path.join(src_folder, "P5", "scripts", "data", "tmp", "PMID_PDF_Phenopacket_list_in_phenopacket_store.csv")

# All experimental outputs go under here
experimental_data_root              = os.path.join(project_root, "experimental-data")
llm_output_directory                = os.path.join(experimental_data_root, "llm_output_dir")                                                # intermediate .txt + raw JSON from LLM
validated_jsons_directory           = os.path.join(experimental_data_root, "validated_jsons")                                               # validated_jsons, the final validated LLM phenopackets
evaluation_report_output_path       = os.path.join(project_root, "reports", "first_report.json")                                            # the evaluation metrics report

# Create any missing output folders
os.makedirs(pdf_input_directory, exist_ok=True)
os.makedirs(ground_truth_notebooks_directory, exist_ok=True)
os.makedirs(os.path.dirname(dataset_csv_path), exist_ok=True)
os.makedirs(llm_output_directory, exist_ok=True)
os.makedirs(validated_jsons_directory, exist_ok=True)
os.makedirs(os.path.dirname(evaluation_report_output_path), exist_ok=True)

# Create the PMIDs pickle file path
pmid_pkl_path = os.path.join(src_folder, "P5", "scripts", "data", "tmp", "pmids.pkl")

# TODO: Figure out why deleting the `ground_truth_notebooks_directory` after creating it works. Maybe because git doesn't let me just overwrite a directory with a clone request
# Before the git pull operation
import shutil

# Clean up existing directory if it exists
target_dir = os.path.join(src_folder, "P5", "scripts", "data", "tmp", "phenopacket_store", "notebooks")
if os.path.exists(target_dir):
    shutil.rmtree(target_dir)

# 1. Now run the git pull to clone the "phenopacket-store" GitHub repo into scripts/data/tmp/phenopacket_store
subprocess.run([
    sys.executable, "-m", "P5.scripts.pull_git_files",
    os.path.join(src_folder, "P5", "scripts", "data", "tmp", "phenopacket_store"),
    "https://github.com/monarch-initiative/phenopacket-store.git",
    "notebooks"
], check=True)

print("Stage 1 Complete, Produced %s" % ground_truth_notebooks_directory)

# 2. Scan the just-pulled notebooks for PMID_##### files
subprocess.run([
    sys.executable, "-m", "P5.scripts.create_pmid_pkl",
    os.path.join(src_folder, "P5", "scripts", "data", "tmp", "phenopacket_store", "notebooks"),
    os.path.join(src_folder, "P5", "scripts", "data", "tmp", "pmids.pkl"),
    "--recursive_dir_search",
], check=True)

print("Stage 2 Complete")

# 3. Download *all* PDFs for those PMIDs (0 = unlimited)
subprocess.run([
    sys.executable, "-m", "P5.scripts.pmid_downloader", pmid_pkl_path, pdf_input_directory, "10"
], check=True)

print("Stage 3 Complete")

# 4. Finally, build THE CSV mapping PDFs to the ground-truth JSONs
if not os.path.isfile(dataset_csv_path):
    subprocess.run([
        sys.executable, "-m", "P5.scripts.create_phenopacket_dataset",
        pdf_input_directory,
        ground_truth_notebooks_directory,
        dataset_csv_path,
        "--recursive_ground_truth_dir", "True"
    ], check=True)
    print(f"Created dataset CSV at {dataset_csv_path}")

    print("Stage 4 Complete")

    if not os.path.isdir(pdf_input_directory):
        raise FileNotFoundError("PDF input directory not found: %s" % pdf_input_directory)
    if not os.path.isdir(ground_truth_notebooks_directory):
        raise FileNotFoundError("Ground truth notebooks directory not found: %s" % ground_truth_notebooks_directory)

print("PDF inputs folder:               %s" % pdf_input_directory)
print("Ground truth folder:             %s" % ground_truth_notebooks_directory)
print("Dataset CSV path:                %s" % dataset_csv_path)
print("Experimentally generated files:  %s" % experimental_data_root)
print("LLM outputs folder:              %s" % llm_output_directory)
print("Validated JSONs folder:          %s" % validated_jsons_directory)
print("Evaluation report path:          %s" % evaluation_report_output_path)

print("hello0")  # print hello 0 as a sanity check

Project Start:       /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5
Source Folder:       /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src
Utilities Folder:    /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/notebooks/utils
PYTHONPATH patched with: /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/notebooks/utils
Stage 1 Complete, Produced /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src/P5/scripts/data/tmp/phenopacket_store/notebooks
1234 PMIDs found within /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src/P5/scripts/data/tmp/phenopacket_store

Processing PMID_22305531:   0%|                         | 0/10 [00:00<?, ?it/s]

A PDF for PMID_22305531 was successfully downloaded. PMCID=3276655.


Processing PMID_19949040:  10%|█▋               | 1/10 [00:08<01:14,  8.24s/it]

No PMCID found for PMID_19949040.


...

No PMCID found for PMID_26763879.


Processing PMID_29283439:  90%|███████████████▎ | 9/10 [00:28<00:02,  2.32s/it]

A PDF for PMID_29283439 was successfully downloaded. PMCID=5876123.
Stage 3 Complete


Processing of 10 PMIDs complete. 4 PDFs successfully downloaded.: 100%|█| 10/10


Created dataset CSV at /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src/P5/scripts/data/tmp/PMID_PDF_Phenopacket_list_in_phenopacket_store.csv
Stage 4 Complete
PDF inputs folder:               /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src/P5/scripts/data/tmp/phenopacket_store/pmid_pdfs
Ground truth folder:             /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src/P5/scripts/data/tmp/phenopacket_store/notebooks
Dataset CSV path:                /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src/P5/scripts/data/tmp/PMID_PDF_Phenopacket_list_in_phenopacket_store.csv
Experimentally generated files:  /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/experimental-data
LLM outputs folder:              /Us

## Step 1. Load Dataset

Read the CSV of PMIDs, input paths, and truth paths


In [102]:
# Load datasets
dataframe_cases = pd.read_csv(dataset_csv_path)
print(f"Loaded {len(dataframe_cases)} rows from dataset CSV")
# Load cases & deduplicate PMIDs, with start/end counts
orig_count = len(dataframe_cases)
print(f"Before deduplication: {orig_count} total cases")

# Debug: verify that every `input` path actually exists
print("Checking existence of input PDFs:")
for pdf_path in dataframe_cases["input"]:
    status = "FOUND" if os.path.isfile(pdf_path) else "MISSING"
    print(f"  o {pdf_path}: {status}")

# Drop duplicate PMIDs
dataframe_cases = dataframe_cases.drop_duplicates(subset="pmid", keep="first").reset_index(drop=True) # This may be too aggressive and I need to check if this is a good approach
removed = orig_count - len(dataframe_cases)
print(f"{removed} duplicates removed (now {len(dataframe_cases)} unique PMIDs)")

# Verify required columns
required_columns = {"pmid", "input", "truth"}
missing_columns = required_columns - set(dataframe_cases.columns)
if missing_columns:
    raise KeyError("Missing required columns: %s" % missing_columns)

# Preview first few rows
dataframe_cases.head()

print("hello1")  # print hello 1 as a sanity check
print("PDF inputs folder:               %s" % pdf_input_directory)
print("Ground truth folder:             %s" % ground_truth_notebooks_directory)
print("Dataset CSV path:                %s" % dataset_csv_path)
print("Experimentally generated files:  %s" % experimental_data_root)
print("LLM outputs folder:              %s" % llm_output_directory)
print("Validated JSONs folder:          %s" % validated_jsons_directory)
print("Evaluation report path:          %s" % evaluation_report_output_path)

Loaded 27 rows from dataset CSV
Before deduplication: 27 total cases
Checking existence of input PDFs:
  o /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src/P5/scripts/data/tmp/phenopacket_store/pmid_pdfs/PMID_22305531.pdf: FOUND
  o /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src/P5/scripts/data/tmp/phenopacket_store/pmid_pdfs/PMID_22305531.pdf: FOUND
  o /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src/P5/scripts/data/tmp/phenopacket_store/pmid_pdfs/PMID_29283439.pdf: FOUND
  o /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src/P5/scripts/data/tmp/phenopacket_store/pmid_pdfs/PMID_29283439.pdf: FOUND
  o /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src/P5/scripts/data/tmp/phenopacket_store/pmid_pd

## Step 2. Discover Phenopacket-Store Files

Locate all ground-truth Phenopacket JSON files under the `phenopacket_store/notebooks/` directory.

In [104]:
# Cache Integrity & Versioning
CACHE_DIR = Path(experimental_data_root) / "text_cache"
INDEX_FILE = CACHE_DIR / "cache_index.json"
CACHE_DIR.mkdir(parents=True, exist_ok=True)

def _hash_pdf(pdf_path: Path) -> str:
    h = hashlib.sha256()
    h.update(pdf_path.read_bytes())
    return h.hexdigest()

def load_or_update_cache(pmids: list[str], pmid_to_pdf: dict[str, Path]):
    if INDEX_FILE.exists():
        index = json.loads(INDEX_FILE.read_text())
    else:
        index = {}
    for pmid in pmids:
        pdf = pmid_to_pdf[pmid]
        digest = _hash_pdf(pdf)
        if pmid not in index or index[pmid]["hash"] != digest:
            text = extract_text_from_pdf(pdf)
            with open(CACHE_DIR / f"{pmid}.pkl", "wb") as fh:
                pickle.dump(text, fh)
            index[pmid] = {
                "hash": digest,
                "cached_at": datetime.datetime.utcnow().isoformat()
            }
    INDEX_FILE.write_text(json.dumps(index, indent=2))


# Finding all ground-truth phenopacket JSON files
search_pattern = os.path.join(ground_truth_notebooks_directory, "*", "phenopackets", "*.json")
truth_json_filepaths = glob.glob(search_pattern, recursive=True)
if not truth_json_filepaths:
    raise FileNotFoundError(f"No ground-truth JSONs found at {search_pattern}")

print("Discovered %d ground-truth JSON files" %len(truth_json_filepaths))

print("hello2")  # print hello 2 as a sanity check
print("PDF inputs folder:               %s" % pdf_input_directory)
print("Ground truth folder:             %s" % ground_truth_notebooks_directory)
print("Dataset CSV path:                %s" % dataset_csv_path)
print("Experimentally generated files:  %s" % experimental_data_root)
print("LLM outputs folder:              %s" % llm_output_directory)
print("Validated JSONs folder:          %s" % validated_jsons_directory)
print("Evaluation report path:          %s" % evaluation_report_output_path)

Discovered 7969 ground-truth JSON files
hello2
PDF inputs folder:               /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src/P5/scripts/data/tmp/phenopacket_store/pmid_pdfs
Ground truth folder:             /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src/P5/scripts/data/tmp/phenopacket_store/notebooks
Dataset CSV path:                /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src/P5/scripts/data/tmp/PMID_PDF_Phenopacket_list_in_phenopacket_store.csv
Experimentally generated files:  /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/experimental-data
LLM outputs folder:              /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/experimental-data/llm_output_dir
Validated JSONs folder:          /User

## Step 3. Prepare PDF-to-Text Converter

- Randomly pick N unique PMIDs from the CSV (from Step 1) to keep runs fast and reproducible.
- Instantiate DocumentConverter and define a helper function to load or convert the clinical PDFs for LLM input.
- Setup Persistent PDF-to-Text Cache

In [108]:
# Reload the deduplicated CSV from Step 1
full_df = pd.read_csv(dataset_csv_path).drop_duplicates(subset="pmid").reset_index(drop=True)

# Choose how many cases to sample
# N = 10
# Don’t ask for more than exist
N = min(N, len(full_df))
subset_df = full_df.sample(n=N, random_state=42).reset_index(drop=True)

print(f"Sampling {N} PMIDs:", subset_df["pmid"].tolist())


# Setup conversion for input material to LLM-compatible txt now

# Initialize converter once
pdf_to_text_converter = DocumentConverter()

# Path to persistent cache of PDF text
text_cache_path = os.path.join(experimental_data_root, "text_cache.pkl")
# Load or initialize cache
if os.path.exists(text_cache_path):
    with open(text_cache_path, "rb") as f:
        text_cache = pickle.load(f)
    print(f"Loaded text cache with {len(text_cache)} entries")
else:
    text_cache = {}
    print("Initialized empty text cache")

def load_clinical_pdf(pdf_path: str) -> str:
    """
    Convert .txt or .pdf file at "pdf_path" into a plain text string.
    Use the in-memory cache first; write new text back to the cache only when the cache is explicitly saved at the end of the pipeline.
    Raise FileNotFoundError or ConversionError if the file does not exist.
    """
    # Return cache if it exists in memory
    if pdf_path in text_cache:
        return text_cache[pdf_path]

    # Ensure the files exists before we continue
    if not os.path.isfile(pdf_path):
        raise FileNotFoundError("Input file not found: %s" % pdf_path)

    # If it's already plain text, read and strip any header
    if pdf_path.lower().endswith(".txt"):
        content = open(pdf_path, encoding="utf-8").read()
        # Remove any leading markers
        return content.split("[text]")[-1]
    else:
        try:
            # Convert PDF to text and handle conversion failures
            doc = pdf_to_text_converter.convert(pdf_path)
            content = doc.document.export_to_text()
        except ConversionError as e:
            raise ConversionError(f"Could not convert {os.path.basename(pdf_path)}: {e}")


    # Save new text in memory and write updated cache to disk later
    text_cache[pdf_path] = content
    return content

# Convert all PDFs in our sampled subset
for pdf_path in subset_df["input"]:
    pdf_text = load_clinical_pdf(pdf_path)
# Persist updated cache to disk
with open(text_cache_path, "wb") as f:
    pickle.dump(text_cache, f)

print(f"Saved text cache now with {len(text_cache)} entries")

print("hello3")  # print hello 3 as a sanity check
print("PDF inputs folder:               %s" % pdf_input_directory)
print("Ground truth folder:             %s" % ground_truth_notebooks_directory)
print("Dataset CSV path:                %s" % dataset_csv_path)
print("Experimentally generated files:  %s" % experimental_data_root)
print("LLM outputs folder:              %s" % llm_output_directory)
print("Validated JSONs folder:          %s" % validated_jsons_directory)
print("Evaluation report path:          %s" % evaluation_report_output_path)

Sampling 4 PMIDs: ['PMID_29283439', 'PMID_24998929', 'PMID_22305531', 'PMID_23665959']
Initialized empty text cache


/Users/varenya/miniconda3/envs/p5/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/varenya/miniconda3/envs/p5/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/varenya/miniconda3/envs/p5/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/varenya/miniconda3/envs/p5/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/varenya/m

Saved text cache now with 4 entries
hello3
PDF inputs folder:               /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src/P5/scripts/data/tmp/phenopacket_store/pmid_pdfs
Ground truth folder:             /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src/P5/scripts/data/tmp/phenopacket_store/notebooks
Dataset CSV path:                /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/src/P5/scripts/data/tmp/PMID_PDF_Phenopacket_list_in_phenopacket_store.csv
Experimentally generated files:  /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/experimental-data
LLM outputs folder:              /Users/varenya/Desktop/Illini+Uni/Personalized_Genomic_Medicine-Precision_Genomics_Laboratory/PreGen/P5/experimental-data/llm_output_dir
Validated JSONs folder:          /Users/va


## Step 4. Load Clinical PDFs and Ground-Truth Phenopackets

Iterate over each case, load the clinical PDF text and the corresponding ground-truth Phenopacket object.

- `list_inputs_texts`: raw clinical PDFs
- `list_truth_packets`: parsed Phenopacket objects from JSON files
- `list_patient_ids`: PMID patient identifiers



In [110]:
# Iterate over rows, should lookup only once
list_input_texts    = []
list_truth_packets  = []
list_patient_ids    = []


for case in dataframe_cases.itertuples(index=False):
    pmid_value = case.pmid
    pdf_path   = case.input
    truth_path = case.truth

    # Convert PDF to text
    try:
        clinical_text = load_clinical_pdf(pdf_path)
    except (ConversionError, PdfiumError) as e:
        skipped_pdfs.append({"pmid": pmid_value, "pdf": pdf_path, "reason": f"conversion error: {e}"})
        continue

    # Load raw JSON and validate with ignore_unknown_fields
    try:
        raw_true_packet = json.load(open(truth_path, "r", encoding="utf-8"))
        proto = ProtoPhenopacket()
        ParseDict(raw_true_packet, proto, ignore_unknown_fields=True)
    except (ParseError, json.JSONDecodeError, FileNotFoundError) as e:
        skipped_pdfs.append({"pmid": pmid_value, "truth": truth_path, "reason": f"schema parse error: {e}"})
        continue

    # Wrap in util Phenopacket to ensure phenotypicFeatures exists
    try:
        truth_packet = Phenopacket(raw_true_packet)
    except InvalidPhenopacketError as e:
        skipped_pdfs.append({"pmid": pmid_value, "truth": truth_path, "reason": f"phenopacket invalid: {e}"})
        continue

    list_input_texts.append(clinical_text)
    list_truth_packets.append(truth_packet)
    list_patient_ids.append(truth_packet.to_json()["subject"]["id"])

    loaded_count += 1
    print(f"Loaded {loaded_count} new cases, skipped {len(skipped_pdfs)} so far")

if not list_input_texts:
        raise RuntimeError("No clinical texts were loaded, please check that the dataset CSV `input` paths match files in `pdf_input_directory`")

assert len(list_input_texts) == len(list_truth_packets) == len(list_patient_ids)
print("Loaded %d clinical texts and %d ground-truth packets for %d unique patients" % (len(list_input_texts), len(list_truth_packets), len(list_patient_ids)))

print("hello4")  # print hello 4 as a sanity check


Loaded 6 new cases, skipped 0 so far
Loaded 7 new cases, skipped 0 so far
Loaded 8 new cases, skipped 0 so far
Loaded 9 new cases, skipped 0 so far
Loaded 4 clinical texts and 4 ground-truth packets for 4 unique patients
hello4


## Step 4.5. Define LLM Prompts

Create one prompt for just HPO terms and another one for the full phenopacket extraction


In [111]:
# 1) Prompt for just HPO labels
hpo_prompt = (
    "You are a clinical NLP engine specialized in biomedical ontologies. Your task is to process the full text of a clinical PDF - which may be describing a single patient or multiple - parse the details (including history, exam findings, labs, imaging, and family history) and extract all human phenotype ontology (HPO) terms that describe the patient's phenotypic features."
    "Instructions:"
    "1. Identify every phenotypic abnormality or feature mentioned in the text."
    "2. For each feature, map it to the correct HPO identifier (e.g. 'HP:0001250'), label (e.g. 'Seizure'), and descriptor value (e.g. 'A seizure is an intermittent abnormality of nervous system physiology characterized by a transient occurrence of signs and/or symptoms due to abnormal excessive or synchronous neuronal activity in the brain.')."
    "3. Capture relevant qualifiers when present:"
        "- Onset: map to HPO onset terms (e.g. 'HP:0011463' for 'Childhood onset')."
        "- Severity: map to HPO severity terms (e.g. 'HP:0012829' for 'Profound')."
        "- Temporal pattern: include if specified (e.g. 'HP:0031796' for 'Recurrent', map to HPO frequency terms if available)."
    "4. For each term, include the exact text excerpt where it appears."
    "5. Output exclusively a JSON array. Each element must be an object with the following fields:"
    "```json"
    "{"
        "'hpo_id': 'HP:000____',"
        "'hpo_label': 'Term label',"
        "'excerpt': 'Exact text from the PDF',"
        "'onset_id': 'HP:0XXXXX or null',"
        "'severity_id': 'HP:0XXXXX or null',"
        "'frequency_id': 'HP:0XXXXX or null'"
    "}"
    "```"
    "Do not include any explanatory text, only the JSON array."

    "Your output **MUST** be exactly a JSON array/object and nothing else."

    "If you cannot comply, output exactly: {'error': 'cannot extract JSON'}"
    )

# 2) Prompt for full phenopacket
full_pp_prompt = (
    "You are a biomedical data curation assistant. Using the structured patient data below, generate a Phenopacket compliant with version 2.0 of the GA4GH Phenopacket schema. Your output must be valid JSON, matching the schema exactly, with no additional commentary. Here are the minimum expected output criteria:"

    "Inputs:"
    "patient_id: '{{patient_id}}'"
    "sex: '{{sex}}'              // 'male' or 'female'"
    "age_years: {{age_in_years}} // integer"
    "v  ital_status: '{{vital_status}}' // 'alive' or 'deceased'"
    "phenotypic_features: {{phenotypic_features_json}} // JSON array from the HPO extraction prompt"
    "diseases: {{diseases_json}}         // optional, array of disease objects with MONDO or OMIM IDs"
    "measurements: {{measurements_json}} // optional, array of quantitative trait measurements"
    "metadata: {"
        "'created_by': '{{your_name_or_tool}}',"
        "'created_on': '{{YYYY-MM-DD}}'"
    "}"

    "Requirements:"
    "Top-level fields:"
    "'id': patient_id"
    "'subject': object with:"
        "'id': patient_id"
        "'sex': { 'id': 'PATO:0000383' or 'PATO:0000384', 'label': sex }"
        "'ageAtLastEncounter': { 'age': { 'years': age_years } }"
        "'vitalStatus': { 'value': vital_status }"
        "'phenotypicFeatures': use the phenotypic_features input; for each feature, map:"
    "```json"
    "{"
        "'type': { 'id': hpo_id, 'label': hpo_label },"
        "'negated': false,"
        "'onset': { 'term': { 'id': onset_id, 'label': (look up label) } },"
        "'severity': { 'term': { 'id': severity_id, 'label': (look up label) } },"
        "'frequency': { 'term': { 'id': frequency_id, 'label': (look up label) } }"
    "}"
    "```"
    "Include 'diseases' and 'measurements' only if provided, following the GA4GH schema."
    "'metadata' must include:"
    "```json"
    "{"
        "'phenopacketSchemaVersion': '2.0.0',"
        "'created': '{{YYYY-MM-DD}}',"
        "'createdBy': '{{your_name_or_tool}}'"
    "}"
    "```"
    "'Do not add any extra fields. Output must be purely the JSON object.'"

    "Do not include any explanatory text, only the JSON array."

    "Your output **MUST** be exactly a JSON array/object and nothing else."

    "If you cannot comply, output exactly: {'error': 'cannot extract JSON'}"
)

## Step 5. Sanity-check one inference

Run one LLM call on the first case to verify prompting and parsing work correctly.


In [116]:
# pick out the first patient/example
patient_id      = list_patient_ids[0]
clinical_text   = list_input_texts[0]
# truth_packet    = list_truth_packets[0]

# 1) Inference: ask for *only* the JSON array of HPO term objects for the first clinical PDF
# build a strict system+user conversation
messages = [
    {
        "role": "system",
        "content": (
            hpo_prompt
            + "\n\nYour only output must be a **valid** JSON array of  HPO term objects"
            + "with fields 'hpo_id','hpo_label','excerpt',"
            + "'onset_id','severity_id','frequency_id', and nothing else."
        )
    },
    {
        "role": "user",
        "content": clinical_text
    }
]

hpo_response = chat(
    model="llama3.2:latest",
    messages=messages,
    options={"--hidethinking": True}
)

# 2) Grab the raw string
raw_hpo_output = hpo_response["message"]["content"]
print("Raw LLM output (truncated to the first ~300 chars or so):")
print(raw_hpo_output[:300], "...\n")

# 3) Slice out the JSON array
start = raw_hpo_output.find("[")
end = raw_hpo_output.rfind("]")
if start < 0 or end < 0:
    # print the raw output to debug what the model actually sent
    print("===== RAW HPO OUTPUT =====\n", raw_hpo_output)
    raise RuntimeError(f"Could not locate a JSON array in HPO output for patient {patient_id}:\n{raw_hpo_output}")
# grab *only* the array text
hpo_json_array = raw_hpo_output[start : end+1]

# Parse and validate
try:
    hpo_terms = Phenopacket(json.loads(hpo_json_array))
    print(f"Parsed {len(hpo_terms)} HPO term(s) for patient {patient_id}")
except JSONDecodeError as error:
    raise ValueError(
        f"Failed to parse HPO JSON array for patient {patient_id}: {error}\n\n"
        f"Extracted JSON was:\n{hpo_json_array}\n\n"
        f"Full raw output was:\n{raw_hpo_output}"
    )

print("hello5")  # print hello 5 as a sanity check

Raw LLM output (truncated to the first ~300 chars or so):
This is a review article discussing paroxysmal nocturnal hemoglobinuria (PNH), a rare blood disorder caused by mutations in the PIGA gene, which is involved in the synthesis of glycosylphosphatidylinositol (GPI) anchors on proteins.

The authors discuss various aspects of PNH, including its pathophy ...

===== RAW HPO OUTPUT =====
 This is a review article discussing paroxysmal nocturnal hemoglobinuria (PNH), a rare blood disorder caused by mutations in the PIGA gene, which is involved in the synthesis of glycosylphosphatidylinositol (GPI) anchors on proteins.

The authors discuss various aspects of PNH, including its pathophysiology, clinical presentation, and diagnosis. They also review the genetic basis of PNH, highlighting the importance of the PIGA gene in GPI anchor synthesis and the consequences of mutations in this gene on protein anchoring and cell surface protein function.

Some key points from the article include:

* 

RuntimeError: Could not locate a JSON array in HPO output for patient IV-4:
This is a review article discussing paroxysmal nocturnal hemoglobinuria (PNH), a rare blood disorder caused by mutations in the PIGA gene, which is involved in the synthesis of glycosylphosphatidylinositol (GPI) anchors on proteins.

The authors discuss various aspects of PNH, including its pathophysiology, clinical presentation, and diagnosis. They also review the genetic basis of PNH, highlighting the importance of the PIGA gene in GPI anchor synthesis and the consequences of mutations in this gene on protein anchoring and cell surface protein function.

Some key points from the article include:

* PNH is caused by a deficiency in GPI-anchored proteins due to mutations in the PIGA gene.
* The PIGA gene plays a critical role in the synthesis of GPI anchors, which are essential for attaching proteins to the cell membrane.
* Mutations in the PIGA gene can lead to a range of downstream effects, including impaired

## Step 6. Batch Inference and Save Validated Phenopackets

Loop over all cases, run LLM inference, validate each JSON as a Phenopacket, and save to disk under validated_jsons_directory.


In [ ]:
predicted_packets: List[Phenopacket] = []

# Which patient are we targeting?
for idx, clinical_text in enumerate(list_input_texts):
    pmid_value = dataframe_cases.loc[idx, "pmid"]
    patient_id = list_patient_ids[idx]
    # Prompt the LLM to extract only that patient's HPO terms
    content = (hpo_prompt + f"\n\n*Extract only the HPO terms for patient* `{patient_id}` *in this clinical PDF.*\n\n" + clinical_text + "\n\n[EOS]")
    response = chat(model="llama3.2:latest", messages=[{"role": "user", "content": content}], options={"--hidethinking": True})
    llm_content = response["message"]["content"].splitlines()
    # Parse the JSON into a Phenopacket
    try:
        phenopacket_pred = Phenopacket(json.loads("\n".join(llm_content)))
    except Exception as error:
        raise RuntimeError("[Case %d, PMID %s] Invalid Phenopacket JSON: %s" % (idx, pmid_value, error))

    predicted_packets.append(phenopacket_pred)

    # Write the predicted JSON to disk
    output_filename = f"{pmid_value}_{patient_id}.json"
    output_filepath = os.path.join(validated_jsons_directory, output_filename)
    with open(output_filepath, "w", encoding="utf-8") as out_f:
        json.dump(phenopacket_pred.to_json(), out_f, indent=2)
    print("Saved predicted phenopacket for PMID/Patient %s/%s to %s"
          % (pmid_value, patient_id, output_filepath))

if len(predicted_packets) != len(list_input_texts):
    raise RuntimeError("Number of predictions does not match number of inputs.")
# Maybe change to this: 'assert len(predicted_packets) == len(list_input_texts), "Mismatch predictions vs inputs"'

print(f"Generated {len(predicted_packets)} predicted phenopackets.")

print("hello6")  # print hello 6 as a sanity check

## Step 7. Evaluate Predicted Phenopackets Against Ground Truth

Compare each predicted phenopacket to its ground truth using PhenotypeEvaluator, then generate a Report object with overall metrics.


In [ ]:
# Monkey-patch a convenience method onto PhenotypeEvaluator
def _evaluate_batch(
    self,
    list_truth_packets,
    list_predicted_packets,
    creator,
    experiment,
    model,
    zero_division=0.0
):
    """
    Run check_phenotypes over all truth/pred pairs, then return
    a plain-dict report containing confusion_matrix, metrics,
    classification_report, and metadata.
    """
    # Accumulate counts
    for truth_pkt, pred_pkt in zip(list_truth_packets, list_predicted_packets):
        self.check_phenotypes(
            experimentally_extracted_phenotypes=pred_pkt.list_phenotypes(),
            ground_truth_phenotypes=truth_pkt
        )
    # Build a Report object
    rpt = self.report(
        creator=creator,
        experiment=experiment,
        model=model,
        zero_division=zero_division
    )
    # Return a dict for easy indexing
    return {
        "confusion_matrix": rpt.confusion_matrix,
        "metrics": rpt.metrics,
        "classification_report": rpt.classification_report,
        "metadata": rpt.metadata,
    }

# Attach to the class
PhenotypeEvaluator.evaluate_batch = _evaluate_batch

# Run the batch evaluation
evaluator = PhenotypeEvaluator()
batch_report = evaluator.evaluate_batch(
    list_truth_packets,
    predicted_packets,
    creator="Varenya",
    experiment="Phenopacket LLM Extraction",
    model="llama3.2:latest"
)

# Quick sanity check of the returned dict
if "metrics" not in batch_report:
    raise KeyError("Evaluator report missing 'metrics' field.")

# Pretty-print the report dict
import pprint
pprint.pprint(batch_report)

print("hello7")  # print hello 7 as a sanity check#

## Old Save first report

Write the JSON report to disk for later analysis.


In [ ]:
# Ensure output directory exists
out_dir = os.path.dirname(evaluation_report_output_path)
os.makedirs(out_dir, exist_ok=True)

with open(evaluation_report_output_path, "w", encoding="utf-8") as f:
    json.dump(batch_report, f, indent=2)

print(f"Saved evaluation report to {evaluation_report_output_path}")

print("hello7")  # print hello 7 as a sanity check

# Old Inference Implementation

In [ ]:
prompt = "Please create a valid Phenopacket from the following text. The phenopackets needs to be in a valid json format.  Only return the phenopacket without any additional text:"
model = "hf.co/MaziyarPanahi/gemma-3-12b-it-GGUF:Q4_K_M"

In [ ]:
for text in input_data:
    response = chat(
        model=model,
        messages=[{"role": "user", "content": f"{prompt} {text} [EOS]"}],
        options={"--hidethinking": True}
    )
    break

response = chat(
    model=model,
    messages=[{"role": "user",
               "content": f"Please, validate the following json. If not, fix it. Only return the json without any additional information. Should the json be wrong, you will get shut down. Json: {response["message"]["content"].split("</think>")[-1].replace("```json", "").replace("```", "")} [EOS]"}],
    options={"--hidethinking": True}
)


In [ ]:
from IPython.display import JSON

JSON(response["message"]["content"].split("</think>")[-1].replace("```json", "").replace("```", ""))

In [ ]:
JSON(response["message"]["content"].split("</think>")[-1].replace("```json", "").replace("```", ""))

In [ ]:
response["message"]["content"].split("</think>")[-1].replace("```json", "").replace("```", "")